In [1]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil
from finn.util.visualization import showInNetron
from finn.util.test import get_example_input
import numpy as np

model_file = "cybsec-mlp-ready.onnx"

rtlsim_output_dir = "output_proj"

#Delete previous run results if exist
if os.path.exists(rtlsim_output_dir):
    # shutil.rmtree(rtlsim_output_dir)
    print("Previous run has to be deleted!")

cfg_stitched_ip = build.DataflowBuildConfig(
    output_dir          = rtlsim_output_dir,
    mvau_wwidth_max     = 1000000, #tinha usado 80
    target_fps          = 1000000, #tinha usado 100
    synth_clk_period_ns = 10.0,
    fpga_part           = "xc7z020clg400-1", 
    rtlsim_batch_size   = 1000,
    stitched_ip_gen_dcp = True,
    board               = "Pynq-Z1",
    shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ]
)

Previous run has to be deleted!


In [ ]:
%%time
build.build_dataflow_cfg(model_file, cfg_stitched_ip)

Building dataflow accelerator from cybsec-mlp-ready.onnx
Intermediate outputs will be generated in................ aqui? acho que sim! ->/home/artti/Desktop/finn/notebooks/sat6_cnn/builds_sources
Final outputs will be generated in output_proj
Build log is at output_proj/build_dataflow.log
Running step: step_qonnx_to_finn [1/18]
Running step: step_tidy_up [2/18]
Running step: step_streamline [3/18]
Running step: step_convert_to_hls [4/18]
Running step: step_create_dataflow_partition [5/18]
Running step: step_target_fps_parallelization [6/18]
Running step: step_apply_folding_config [7/18]
Running step: step_minimize_bit_width [8/18]
Running step: step_generate_estimate_reports [9/18]
Running step: step_hls_codegen [10/18]
Running step: step_hls_ipgen [11/18]
Running step: step_set_fifo_depths [12/18]
Running step: step_create_stitched_ip [13/18]
Running step: step_measure_rtlsim_performance [14/18]
Running step: step_out_of_context_synthesis [15/18]
Running step: step_synthesize_bitfile 

ERROR: [Common 17-161] Invalid option value '' specified for 'objects'.
Traceback (most recent call last):
  File "/home/artti/Desktop/finn/src/finn/builder/build_dataflow.py", line 158, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/home/artti/Desktop/finn/src/finn/builder/build_dataflow_steps.py", line 757, in step_synthesize_bitfile
    model = model.transform(
  File "/home/artti/Desktop/finn/deps/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/home/artti/Desktop/finn/src/finn/transformation/fpgadataflow/make_zynq_proj.py", line 349, in apply
    model = model.transform(MakeZYNQProject(self.platform, enable_debug=self.enable_debug))
  File "/home/artti/Desktop/finn/deps/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/home/artti/Desktop/finn/src/fi

> /home/artti/Desktop/finn/src/finn/transformation/fpgadataflow/make_zynq_proj.py(268)apply()
    266         bitfile_name = vivado_pynq_proj_dir + "/finn_zynq_link.runs/impl_1/top_wrapper.bit"
    267         if not os.path.isfile(bitfile_name):
--> 268             raise Exception(
    269                 "Synthesis failed, no bitfile found. Check logs under %s" % vivado_pynq_proj_dir
    270             )



In [ ]:
assert os.path.exists(rtlsim_output_dir + "/report/ooc_synth_and_timing.json")
assert os.path.exists(rtlsim_output_dir + "/report/rtlsim_performance.json")
assert os.path.exists(rtlsim_output_dir + "/final_hw_config.json")

In [ ]:
#Testar agora a compatibilidade entre os resultados e pegar a acurácia

In [ ]:
# import numpy as np

# data_array = test_quantized_dataset.tensors[0][:n_verification_inputs].numpy();

# # Número de bits em cada conjunto
# input_bits = 400
# bits_per_set = 40
# set_numbers = 15
# clock_period = 20
# print("Imprimindo arrays")
# print("-----------------")
# print("s_axis_0_tvalid = 1;")
# print("m_axis_0_tready = 1;")
# for data in data_array:
#     # Dividir o array em conjuntos de x bits
#     sets = [data[i:i + bits_per_set] for i in range(0, len(data), bits_per_set)]
#     # Formatar a saída
#     for i, conjunto in enumerate(sets):
#         # Converter o conjunto para uma string de 40 bits
#         conjunto = (conjunto+1)/2
#         bit_string = ''.join(map(str, conjunto.astype(int)))
#         # Imprimir a saída formatada

#         if (i%(set_numbers) == 0):
#             print(f"s_axis_0_tdata = {bits_per_set}'b{bit_string[::-1]};")
#         else:
#             print(f"#{clock_period} s_axis_0_tdata = {bits_per_set}'b{bit_string[::-1]};")
#     print(f"#{clock_period}")
#     print("s_axis_0_tvalid = 0;")
#     print(f"s_axis_0_tdata = {bits_per_set}'b0_0;")    
#     print(f"#{100*clock_period*set_numbers}")
#     print("s_axis_0_tvalid = 1;")

    
# print(f"#{clock_period}")
# print(f"s_axis_0_tdata = {bits_per_set}'b0_0;") 
# print("s_axis_0_tvalid = 0;")
# print("m_axis_0_tready = 0;")
# print("----------------------------")
# print("Nro de entradas:" + str(len(data_array)))

In [ ]:
# import numpy
# import os
# import re

# caminho_da_pasta = "/home/afely/Desktop/finn-notebook/notebooks/end2end_example/cybersecurity/output_ipstitch_ooc_rtlsim/verification_output"

# arquivos_na_pasta = os.listdir(caminho_da_pasta)

# array_outputs = numpy.zeros(len(arquivos_na_pasta))

# # Itera sobre cada arquivo
# for nome_arquivo in arquivos_na_pasta:
#     caminho_arquivo = os.path.join(caminho_da_pasta, nome_arquivo)
#     output = numpy.load(caminho_arquivo)
#     padrao = re.compile(r"_([0-9]+)_")
#     correspondencia = padrao.search(nome_arquivo)    
#     numero_do_teste = int(correspondencia.group(1))
#     array_outputs[numero_do_teste] = output[0]

# array_outputs_base = output_tensor.numpy()

# if(len(array_outputs) == len(array_outputs_base)):
#     sucess = 0
#     for i in range(len(array_outputs)):
#         if (array_outputs[i] == array_outputs_base[i]):
#             sucess = sucess + 1
#             if(array_outputs[i] == 1):
#                 print("(1) Igual em " + str(i))
#             else:
#                 print("(0) Igual em " + str(i))                
#         else:
#             if(array_outputs[i] == 1):
#                 print("(1) Diferença em " + str(i))
#             else:
#                 print("(0) Diferença em " + str(i)) 


#     precision = sucess/len(array_outputs)
#     print(precision)

# #ele ta comparando estranho
# #o meu array outuputs ta gerando -1 ao inves de 0

In [ ]:
# model_file = "/home/finn/Desktop/finn/notebooks/end2end_example/cybersecurity/output_ipstitch_ooc_rtlsim/intermediate_models/step_create_stitched_ip.onnx"
# showInNetron(model_file)